In [17]:
import nltk
import random
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
import json
import pickle
from sklearn.metrics import accuracy_score, f1_score

In [18]:
label_map={'Aspect1':1 ,'Aspect2':2 ,'Aspect3':3 ,'Aspect4':4 ,'Aspect5':5,'Aspect6':6 ,'Aspect7':7}

In [19]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,label_map[directory]) for sentence in data_temp]
    return dataset

In [20]:
def get_words(x):    #returns all the words in the  data set
    vocab = []
    for sentence,polarity in x:  
        vocab.extend(sentence.split())
    return vocab

In [33]:
#execution
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)
with open('vocab_unigram') as f:
    vocab=json.load(f)

In [24]:
#bigrams
print labeled_data[:10]

[(u'work make awesom place', 7), (u'went offic even weekend extra pay', 2), (u'work life balanc', 4), (u'worker friendli', 3), (u'learn lot new thing', 1), (u'manag suck fire mostli everyon noth', 3), (u'enjoy job', 7), (u'met great peopl work', 7), (u'amaz place work', 7), (u'know grown opportun', 1)]


In [7]:
print len(labeled_data)

201378


In [9]:
print len(vocab)

6544


In [34]:
vectorizer = CountVectorizer(vocabulary=vocab,min_df=1)
corpus=[sentence for sentence,pol in labeled_data]
features=vectorizer.fit_transform(corpus[:60000])
labels=[label for sentence,label in labeled_data[:60000]]

In [26]:
classifier=svm.SVC()
#classifier.kernel='linear'

In [27]:
classifier.fit(features[:30000],labels[:30000])
with open('svc_unigram_model','w') as f:
    pickle.dump(classifier,f)

In [35]:
#accuracy
with open('svc_unigram_model') as f:
    classifier=pickle.load(f)
features_test=vectorizer.fit_transform(corpus[50000:55000])
predicted = classifier.predict(features_test)
accuracy=accuracy_score(labels[50000:55000],predicted)
with open('accuracy_svc_unigram','w') as f:
    json.dump(accuracy,f) 

In [36]:
accuracy

0.50039999999999996

In [21]:
dist = classifier.prob_classify(extract_features(sen.split()))
for label in dist.samples():
    print("%s: %f" % (label, dist.prob(label)))

neg: 0.585660
sli_pos: 0.035267
sli_neg: 0.230902
neutral: 0.123239
pos: 0.024931


In [22]:
polarity=0
for label in dist.samples():
    print("%s: %f" % (label, dist.prob(label)))
    polarity=max(polarity,dist.prob(label))
print polarity

neg: 0.585660
sli_pos: 0.035267
sli_neg: 0.230902
neutral: 0.123239
pos: 0.024931
0.585660041903
